In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import seaborn as sb
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from numba import vectorize
import unidecode
from nltk.corpus import stopwords
import time
import spacy
import re

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.metrics import classification_report



import string,lxml,bs4,nltk
from warnings import simplefilter

In [2]:
contractions = {
"ain't": "are not",
"aren't": "am not",
"can't": "can not",
"can't've": "can not have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I had",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [3]:
def load_and_prapare(fileName):
    
    # READ FILE TO DATAFRAME
    result = pd.read_csv(fileName)
    
    # Remove un necessary columns
    columns_needed = ['ProductId','Score','Summary','Text']
    to_drop_columns = list(x for x in result.columns.tolist() if x not in columns_needed)
    result.drop(to_drop_columns,axis=1,inplace=True)
    
    # Remove Nulls
    columns = ['Summary']
    for column in columns:
        result[column].fillna('', inplace=True)
     
    # Merge Summary and Text
    result['review'] = result['Summary'] + ' ' + result['Text'] 
    result.drop(['Summary','Text','ProductId'],axis=1,inplace=True)
    
    # Remove Outliers
    result['length'] = result['review'].apply(len)
    result[result['length'] < 3000]
    result.drop(['length'],axis=1,inplace=True)
    
    result.drop_duplicates(keep="first",inplace=True)
    
    return result


In [4]:
def load_and_prapare_test(fileName):
    
    # READ FILE TO DATAFRAME
    result = pd.read_csv(fileName)
    
     
    # Merge Summary and Text
    result['review'] = result['Summary'] + ' ' + result['Text'] 
    result.drop(['Summary','Text','ProductId'],axis=1,inplace=True)
    
   
    
    return result


In [5]:
def text_processing(text):
    
    # Transform to Lower characters
    text = text.lower()
    
    # Remove html tags
    sp = BeautifulSoup(text, "html.parser")
    text = sp.get_text(separator=" ")
    
    # Remove accent
    text = unidecode.unidecode(text)
    
    # Remove pucntuations
    for punc in string.punctuation.replace("'",""):
        if punc in text:
            text = text.replace(punc," ")
    
    # Remove Spaces
    text = " ".join(text.split())
    
    # Expand short words
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    
    # Replace '
    text = text.replace("'", " ")
    
    # Remove Spaces
    text = " ".join(text.split())
    
    # Remove stopwords
    sWords = set(stopwords.words('english'))
    sWords = list(sWords)
    deselect_stop_words = ['no', 'not']

    for item in deselect_stop_words:
        sWords.remove(item)
    for word in sWords:
        text = text.replace(" " + word.lower() + " "," ")
        
    # Remove Spaces
    text = " ".join(text.split())
    
    text = re.sub("\S*\d\S*", "", text).strip()
         
    # Lemmatize words spacy       
    #nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    #doc = nlp(text)
    #lemmed = [token.lemma_ for token in doc]
    #lemmed = ' '.join(lemmed) 
    
    # Lemmatize words wordnet
    #lemmatizer = WordNetLemmatizer()
    #lemmed = [lemmatizer.lemmatize(word) for word in text.split()]
    #lemmed = ' '.join(lemmed) 
    
    
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in text.split()]
    stemmed = ' '.join(stemmed) 
       
            
    # Tokenize
    #tokenized = word_tokenize(stemmed)
    
    #return tokenized
    return text
    

In [6]:
#Loading and preparing test data
original_df = load_and_prapare('train.csv')

In [ ]:
#Applying text normalization on training data
original_df['review'] = original_df['review'].apply(text_processing) 

In [ ]:
#Loading test Data
df_test_data = pd.read_csv("test.csv",usecols=('Summary','Text'))
df_test_labels = pd.read_csv("labels.csv",usecols=['Score'])
df_test_data['review'] = df_test_data['Summary'] + ' ' + df_test_data['Text'] 
df_test_data.drop(['Summary','Text'],axis=1,inplace=True)
df_test_data['review'] = df_test_data['review'].fillna('')


In [ ]:
#Applying text normalization on test data
df_test_data['review'] = df_test_data['review'].apply(text_processing)

In [ ]:
df_label = original_df['Score']
df_data = original_df['review']

### with Grid Search

In [ ]:
#making sure we have the same row count on both test reviews and test labels after applying normalization
len(df_data),len(df_label)

In [ ]:
#Splitting training data 
X_train, X_test, y_train, y_test = train_test_split(df_data, df_label, test_size=0.25, random_state=42)

In [ ]:
estimators = [LogisticRegression]

In [ ]:
def create_pipeline(estimator):
    
    pipeline_steps = [
        
        ('cv',CountVectorizer()),
        ('tfidf',TfidfTransformer()),
        ('classifier',estimator)
    ]
    
    return Pipeline(pipeline_steps)

In [ ]:
#  parameters for Grid Search (CV and TfIdf) - BASE 
param_grid= {}

#Parameters for CountVectorizer
param_grid.update({'cv__ngram_range':[(1,2)]})
#param_grid.update({'cv__stop_words':[None,'english']})
#param_grid.update({'cv__max_df':[1,2,3,4]})
#param_grid.update({'cv__analyzer':['word']})


#Parameters of TFIDF
#param_grid.update({'tfidf__use_idf':[True,False]})

# Parameters for LR
param_grid_LR = {'classifier__class_weight' :[{1:0.64 , 2:0.14 , 3:0.08 , 4:0.05 , 5:0.09}]}
#param_grid_LR = {'classifier__C':[1]}
param_grid_LR = {}
param_grid_LR.update(param_grid)

# Parameters for MN
param_grid_MN = {}
#param_grid_MN = {'classifier__alpha':[0.0001,0.001]}
param_grid_MN.update(param_grid)

#Parameters for SGD here
#param_grid_SG={'classifier__loss':['hinge']}
#param_grid_SG.update(param_grid)


In [ ]:
estimators = {
    LogisticRegression : param_grid_LR
}

In [ ]:
def classification_report_final (model): 
    y_pred_final = model.predict(df_test_data['review'])
    score_final =accuracy_score(df_test_labels,y_pred_final)
    print('Model score on unseen data',score_final)
    print (classification_report(df_test_labels,y_pred_final))

In [ ]:
#Building pipeline and applying GridsearchCV 
for estimator,search_param in estimators.items():
    scores=[]
    model = create_pipeline(estimator())
    search=GridSearchCV(model,search_param,n_jobs=-1)
    search.fit(X_train,y_train)
    y_pred = search.predict(X_test)
    score=accuracy_score(y_test,y_pred)
    scores.append(score)
    print(estimator.__name__,'scored',score)
    print("Best parameter (CV score=%0.3f):" % search.best_score_)
    print(search.best_params_)
    print (classification_report(y_test,y_pred))
    print('\n')
    print('Cross validation with unseen test data')
    #classification_report_final(search)

In [ ]:
classification_report_final(search)

LogisticRegression scored 0.750908832166837
Best parameter (CV score=0.751):
{'cv__ngram_range': (1, 2)}
              precision    recall  f1-score   support

           1       0.67      0.71      0.69      7113
           2       0.44      0.18      0.25      4039
           3       0.49      0.33      0.39      5872
           4       0.51      0.29      0.37     11024
           5       0.81      0.96      0.88     49249

    accuracy                           0.75     77297
   macro avg       0.59      0.49      0.52     77297
weighted avg       0.71      0.75      0.72     77297



Cross validation with unseen test data

Model score on unseen data 0.7706559522636757
              precision    recall  f1-score   support

           1       0.71      0.75      0.73     13075
           2       0.58      0.25      0.35      7416
           3       0.56      0.38      0.46     10647
           4       0.57      0.32      0.41     20346
           5       0.82      0.96      0.89     90630

    accuracy                           0.77    142114
   macro avg       0.65      0.53      0.57    142114
weighted avg       0.74      0.77      0.74    142114